In [1]:
import os
import numpy as np
import sys
import json

In [4]:
def process(example, step: int):
    function_key = "Function: "
    observe_key = "Observation: "
    answer_key = "Final Answer: "
    new_data = {}
    if len(example) <= 3:
        new_data["instruction"] = str(example[1]["content"])
        new_data["input"] = ""
        new_data["output"] = answer_key + str(example[2]["content"])
    else:
        name = example[2]["function_call"]["name"]
        args = example[2]["function_call"]["arguments"]
        args = json.loads(str(example[2]["function_call"]["arguments"]))
        for (key, val) in args.items():
            function_key += name + '(' + key + '=' + '"' + val + '"' + ')'
        
        function = function_key
        observe = observe_key + str(example[3]["content"])
        answer = str(example[4]["content"])
        
        if step == 0:
            new_data["instruction"] = str(example[1]["content"])
            new_data["input"] = ""
            new_data["output"] = f"{function}\nObservation: "
        else:
            new_data["instruction"] = str(example[1]["content"])
            new_data["input"] = function + '\n' + observe + '\n' + answer_key
            new_data["output"] = answer
            
    return new_data

    

In [13]:
out_dir = "/home/minhvn/workspace/llm/rikkei_data/instructions-2-eval"

def count_files(folder_path):
    # List all files and directories in the specified folder
    items = os.listdir(folder_path)
    # Use list comprehension to count files (regular files) in the folder
    num_files = len([item for item in items if os.path.isfile(os.path.join(folder_path, item))])
    return num_files

num_files = count_files(out_dir)
print(f"Number of total files: {num_files}")
dataset = []

for idx in range(num_files):
    data_path = os.path.join(out_dir, f'{idx}.json')
    example = json.load(open(data_path))
    for step in range(2):
        if len(example) <= 3 and step == 1:
            continue
        dataset.append(process(example, step))


Number of total files: 77


In [14]:
with open('rikkei_eval.json', 'w', encoding='utf8') as json_file:
    json.dump(dataset, json_file, ensure_ascii=False)

In [15]:
print(len(dataset))

146
